### 【問題1】コードレビュー

#### 1-1
#### clean-workflow-in-keras

In [ ]:
def conv_block(m, dim, acti, bn, res, do=0):
    n = Conv2D(dim, 3, activation=acti, padding='same')(m)
    n = BatchNormalization()(n) if bn else n
    n = Dropout(do)(n) if do else n
    n = Conv2D(dim, 3, activation=acti, padding='same')(n)
    n = BatchNormalization()(n) if bn else n
    return Concatenate()([m, n]) if res else n

conv_blockは同じレベルでの畳み込みをまとめている

mはinputまたは入力レイヤー, dimは出力するチャンネル数, actiは活性化関数, bnはバッチノーマライゼーションするかどうか,
resはResidualNetならTrue,doはdropoutの割合

3×3の畳み込み→バッチノーマライゼーション(bn=Trueの時)→dropout(doが入力されたとき)を2セット行い、

res=Trueなら入力レイヤーであるmと結合する

In [ ]:
def level_block(m, dim, depth, inc, acti, do, bn, mp, up, res):
    if depth > 0:
        n = conv_block(m, dim, acti, bn, res)
        m = MaxPooling2D()(n) if mp else Conv2D(dim, 3, strides=2, padding='same')(n)
        # depthが0になるまでlevel_blockを呼び出す
        m = level_block(m, int(inc * dim), depth - 1,
                        inc, acti, do, bn, mp, up, res)
        if up:
            m = UpSampling2D()(m)
            m = Conv2D(dim, 2, activation=acti, padding='same')(m)
        else:
            m = Conv2DTranspose(dim, 3, strides=2,
                                activation=acti, padding='same')(m)
        n = Concatenate()([n, m])
        m = conv_block(n, dim, acti, bn, res)
    else:
        m = conv_block(m, dim, acti, bn, res, do)
    return m


level_blockは収縮パスと拡張パスを行いU-netの根幹をなしている

エンコーダーは初めにconvblockで畳み込みを行いmaxpoolingで特徴を2倍にする.この処理を再帰的に行いdepthが0になるまで行う.

デコーダーは
up=Trueならupsamplingを行い,Falseならtransposed convolutionを行い、スキップ接続を結合して、畳み込みを行う.

In [ ]:
def UNet(params):

    img_shape = params['input_dim']
    out_ch = 1
    start_ch = 8
    depth = 3
    inc_rate = 2.
    activation = 'relu'
    dropout = 0.5
    batchnorm = False
    maxpool = True
    upconv = True
    residual = False

    i = Input(shape=img_shape)
    o = level_block(i, start_ch, depth, inc_rate, activation,
                    dropout, batchnorm, maxpool, upconv, residual)
    o = Conv2D(out_ch, 1)(o)
    # Sigmoid activation is used because model is trained with binary_crossentropy.
    o =  Activation('sigmoid')(o)

    model = Model(inputs=i, outputs=o)
    
    return model

Unet関数はmodelを生成している

i はinput, oはlevel_blockを使いエンコーダーとデコーダーを実行
oはoutputで最後にsigmoid(2値分類)を行う


#### 1-2 
#### 03-models_pretrained_and_more

In [ ]:
from keras.applications.resnet50 import ResNet50, preprocess_input
# Base model - encoder

    base_model = ResNet50(
        input_shape=input_size, 
        include_top=False,
        weights=weights)

ResNetをダウンロードして転移学習する

encoderのbasemodelをresnetに指定

In [ ]:
# Layers for feature extraction in the encoder part
    encoder1 = base_model.get_layer('activation_1').output
    encoder2 = base_model.get_layer('activation_10').output
    encoder3 = base_model.get_layer('activation_22').output
    encoder4 = base_model.get_layer('activation_40').output
    encoder5 = base_model.get_layer('activation_49').output


encoderに使用するレイヤーはResNetのMaxPooling前のレイヤーとする

encoderは5層になっている

In [ ]:
def decoder_block_simple(
        layer_name, block_name,
        num_filters=32,
        conv_dim=(3, 3)):

    x_dec = Conv2D(
        num_filters, conv_dim,
        padding='same',
        name='{}_conv'.format(block_name))(layer_name)
    x_dec = BatchNormalization(
        name='{}_bn'.format(block_name))(x_dec)
    x_dec = PReLU(
        name='{}_activation'.format(block_name))(x_dec)

    return x_dec

decoder_block_function

upsamplingの後の畳み込みを行う

In [ ]:
# Center block and decorder block
    center = decoder_block(
        encoder5, 'center', num_filters=512)
    concat5 = concatenate([center, encoder5], axis=-1)
ecoder4 = decoder_block(
        concat5, 'decoder4', num_filters=256)
    concat4 = concatenate([UpSampling2D()(decoder4), encoder4], axis=-1)

    decoder3 = decoder_block(
        concat4, 'decoder3', num_filters=128)
    concat3 = concatenate([UpSampling2D()(decoder3), encoder3], axis=-1)

    decoder2 = decoder_block(
        concat3, 'decoder2', num_filters=64)
    concat2 = concatenate([UpSampling2D()(decoder2), encoder2], axis=-1)

    decoder1 = decoder_block(
        concat2, 'decoder1', num_filters=64)
    concat1 = concatenate([UpSampling2D()(decoder1), encoder1], axis=-1)

    # Final upsampling and decoder block for segmentation.
    output = UpSampling2D()(concat1)
    output = decoder_block(
        output, 'decoder_output', num_filters=32)
    output = Conv2D(
        1, (1, 1), activation=None, name='prediction')(output)
    if not use_lovash:
        output = Activation('sigmoid')(output)

centerからdecoderにかけて

基本的にUpsampling後にconcatしてdecoder_functionを使用する

centerだけUpsamplingする前にconcatしている.

outputではlossがロバストヒンジロス以外はバイナリクロスエントロピーのためにsigmoidしている

### 【問題2】コードの書き換え

詳細は03-models_pretrained_and_more-vgg16.ipynbにて

変更したのが

import をVGGにし、

base_modelをVGGにし、

関数名をunet-vgg16に変更し、使用したレイヤーをVGG16のmaxpooling前の

'block2_conv2', shape: (None, 112, 112, 128)

'block3_conv3', shape: (None, 56, 56, 256)

'block4_conv3', shape: (None, 28, 28, 512)

'block5_conv3', shape: (None, 14, 14, 512)

4層にした.decorderの名称も合うように変更した

#### 【問題3】学習・推定
ResNetとVGG双方のコードで学習・推定を行い、結果を比較してください。

epochを20,batch_size16の設定で

ResNetのIOUはBest IoU: 0.6910 at threshold: 0.860であり,

VGG16のIOUはBest IoU: 0.6668 at threshold: 0.840であり

ResNetの方が精度が高く、分類精度が高いほどセグメンテーション精度も高い可能性が示唆
された